# Compare Chunking method & size
1. In some scenarios, Semantic Splitter does not improve the quality of retriever, compared with rudely split. 
2. Hierarchical splitter reserve more information in graph
3. Size of chunking should be suitable to the scenarios (question + content)

In [3]:
!pip install python-dotenv
!pip install llama-index
!pip install wget
!pip install llama-index-vector-stores-azureaisearch
!pip install  azure-search-documents==11.4.0
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai
!pip install EbookLib html2text

  Using cached llama_index_embeddings_azure_openai-0.1.6-py3-none-any.whl.metadata (803 bytes)
  Using cached llama_index_llms_azure_openai-0.1.5-py3-none-any.whl.metadata (786 bytes)
  Using cached azure_identity-1.15.0-py3-none-any.whl.metadata (75 kB)
  Using cached msal-1.28.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.1.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached cffi-1.16.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (1.5 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl.metadata (8.5 kB)
Using cached llama_index_embeddings_azure_openai-0.1.6-py3-none-any.whl (3.0 kB)
Using cached llama_index_llms_azure_openai-0.1.5-py3-none-any.whl (4.5 kB)
Using cached azure_identity-1.15.0-py3-none-any.whl (164 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 1.3 MB/s eta 0:00:0000:0100:020m
Using cached msal-1.28.0-py3-none-any.whl (102 kB)
Using cached msal_extensions-1.1.0-py3-none-an

In [1]:
from azureresource import (
    get_llm,
    get_embed_model,
    get_vector_store
)
from index import build_index, get_index
from llama_index.readers.file.epub import EpubReader
from llama_index.core.node_parser import SentenceSplitter,TokenTextSplitter, HierarchicalNodeParser
tsp = TokenTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
    separator=" ",
)
ssp = SentenceSplitter(chunk_size=256, chunk_overlap=20)
hsp = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)
document = EpubReader().load_data("data/book.epub")

index_dict = {}
llm = get_llm("gpt-35-turbo", "gpt-35-turbo-1106")
embed_model = get_embed_model("text-embedding-ada-002", "text-embedding-ada-002")
def create_index(chunk_size:int, index_name:str):
    index_name = f"chunk-{chunk_size}"
    vector_store = get_vector_store(index_name)
    ssp = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=20)
    index = build_index(vector_store, llm, embed_model, ssp, document)
    return index


/Users/zhuzhirui/vscodeworkspace/Travel/.venv/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [17]:
tnode = tsp.get_nodes_from_documents(document)
[n.to_dict()['text'] for n in tnode[:5]]

['![cover](../Images/cover.jpg)\n\n\n# 版权\n\n版权信息\n\n〔精〕现代文学–031\n\n西班牙旅行笔记\n\n林达◎著\n\n设计：近 浔\n\n排版：林阔阔\n\n* * *\n\n出版：生活·讀書·新知三联书店\n\n开本：720×1280／768×1024\n\n印数：电子书册，不限量。\n\n* * *\n\n声明：本书仅供个人学习之用，请勿用于商业用途。\n\n如对本书产生兴趣，请购买正版书籍。\n\n任何对本书籍的修改、加工、传播，请自负法律后果。\n\n![0](../Images/0.png)\n\n![logo](../Images/logo.png)\n\n与阿獴一起品味经典文学![](../Images/note.png)\n\n  1.',
 '1. ◎请关注微信公众号：阿獴读书，以获取更多精品书籍。\u200b',
 '本书记述了作者在西班牙旅行的经历。面对西班牙既充满异域风情而又绚丽多姿的历史、艺术、人物和宫殿、教堂、城堡等文化精华，作者在描述其无与伦比的人文及艺术价值外，更把上千年来发生在这块土地上的故事，糅进漫游的行程，使帝王将相、战火烽烟、山川景物，尤其是它走向现代国家的进程贯穿于全书，不但能让读者领略西班牙迷人的风貌，更能深入西班牙幽深的历史，洞悉它深刻而富于启示的社会演化过程。',
 '![image](../Images/001.jpeg)\n\n\n![image](../Images/002.jpeg)\n\n\n# 去西班牙\n\n黑色龙卷风袭击我们居住的小镇\n\n我们起程去西班牙旅行\n\n不知怎么，“西班牙”三个字，从小的感觉，就说不清、道不明。在塞万提斯的笔下，西班牙犹如透着阳光的土制酒，谜一般醉人的混合物，在恍惚中令人心驰目眩。可是长大以后，这种感觉，被一些硬邦邦的词击中。那是海明威的书《告别了武器》、《丧钟为谁而鸣》，也是毕加索的画《格尔尼卡》，后面是',
 '也是毕加索的画《格尔尼卡》，后面是“西班牙内战”、“国际纵队”。可是，这两种印象怎么也调和不到一起，也难以相互替代。西班牙就这样模模糊糊地在心里收藏，留在一个角落里，变成了一个隐隐的谜。\n\n其实，我们去过巴塞罗那。']

In [18]:
snode = ssp.get_nodes_from_documents(document)
[n.to_dict()['text'] for n in snode[:5]]

['![cover](../Images/cover.jpg)',
 '![cover](../Images/cover.jpg)\n\n\n# 版权\n\n版权信息\n\n〔精〕现代文学–031\n\n西班牙旅行笔记\n\n林达◎著\n\n设计：近 浔\n\n排版：林阔阔\n\n* * *\n\n出版：生活·讀書·新知三联书店\n\n开本：720×1280／768×1024\n\n印数：电子书册，不限量。\n\n* * *\n\n声明：本书仅供个人学习之用，请勿用于商业用途。\n\n如对本书产生兴趣，请购买正版书籍。\n\n任何对本书籍的修改、加工、传播，请自负法律后果。\n\n![0](../Images/0.png)\n\n![logo](../Images/logo.png)\n\n与阿獴一起品味经典文学![](../Images/note.png)\n\n  1. ◎请关注微信公众号：阿獴读书，以获取更多精品书籍。\u200b',
 '本书记述了作者在西班牙旅行的经历。面对西班牙既充满异域风情而又绚丽多姿的历史、艺术、人物和宫殿、教堂、城堡等文化精华，作者在描述其无与伦比的人文及艺术价值外，更把上千年来发生在这块土地上的故事，糅进漫游的行程，使帝王将相、战火烽烟、山川景物，尤其是它走向现代国家的进程贯穿于全书，不但能让读者领略西班牙迷人的风貌，更能深入西班牙幽深的历史，洞悉它深刻而富于启示的社会演化过程。',
 '![image](../Images/001.jpeg)\n\n\n![image](../Images/002.jpeg)\n\n\n# 去西班牙\n\n黑色龙卷风袭击我们居住的小镇\n\n我们起程去西班牙旅行\n\n不知怎么，“西班牙”三个字，从小的感觉，就说不清、道不明。在塞万提斯的笔下，西班牙犹如透着阳光的土制酒，谜一般醉人的混合物，在恍惚中令人心驰目眩。可是长大以后，这种感觉，被一些硬邦邦的词击中。',
 '那是海明威的书《告别了武器》、《丧钟为谁而鸣》，也是毕加索的画《格尔尼卡》，后面是“西班牙内战”、“国际纵队”。可是，这两种印象怎么也调和不到一起，也难以相互替代。西班牙就这样模模糊糊地在心里收藏，留在一个角落里，变成了一个隐隐的谜。\n\n其实，我们去过巴塞罗那。\n\n那是在冬天，在造访法国的时候，从巴黎直下南方，又从法国南方的

In [20]:
hnode = hsp.get_nodes_from_documents(document)
[n.to_dict()['text'] for n in hnode[:5]]

['![cover](../Images/cover.jpg)\n\n\n# 版权\n\n版权信息\n\n〔精〕现代文学–031\n\n西班牙旅行笔记\n\n林达◎著\n\n设计：近 浔\n\n排版：林阔阔\n\n* * *\n\n出版：生活·讀書·新知三联书店\n\n开本：720×1280／768×1024\n\n印数：电子书册，不限量。\n\n* * *\n\n声明：本书仅供个人学习之用，请勿用于商业用途。\n\n如对本书产生兴趣，请购买正版书籍。\n\n任何对本书籍的修改、加工、传播，请自负法律后果。\n\n![0](../Images/0.png)\n\n![logo](../Images/logo.png)\n\n与阿獴一起品味经典文学![](../Images/note.png)\n\n  1. ◎请关注微信公众号：阿獴读书，以获取更多精品书籍。\u200b\n\n\n本书记述了作者在西班牙旅行的经历。面对西班牙既充满异域风情而又绚丽多姿的历史、艺术、人物和宫殿、教堂、城堡等文化精华，作者在描述其无与伦比的人文及艺术价值外，更把上千年来发生在这块土地上的故事，糅进漫游的行程，使帝王将相、战火烽烟、山川景物，尤其是它走向现代国家的进程贯穿于全书，不但能让读者领略西班牙迷人的风貌，更能深入西班牙幽深的历史，洞悉它深刻而富于启示的社会演化过程。\n\n\n![image](../Images/001.jpeg)\n\n\n![image](../Images/002.jpeg)',
 '![image](../Images/001.jpeg)\n\n\n![image](../Images/002.jpeg)\n\n\n# 去西班牙\n\n黑色龙卷风袭击我们居住的小镇\n\n我们起程去西班牙旅行\n\n不知怎么，“西班牙”三个字，从小的感觉，就说不清、道不明。在塞万提斯的笔下，西班牙犹如透着阳光的土制酒，谜一般醉人的混合物，在恍惚中令人心驰目眩。可是长大以后，这种感觉，被一些硬邦邦的词击中。那是海明威的书《告别了武器》、《丧钟为谁而鸣》，也是毕加索的画《格尔尼卡》，后面是“西班牙内战”、“国际纵队”。可是，这两种印象怎么也调和不到一起，也难以相互替代。西班牙就这样模模糊糊地在心里收藏，留在一个角落里，变成了一个隐隐的谜。\n\n其实，我们去过巴塞罗那。\n\n

In [25]:
[tnode[1].to_dict(), snode[1].to_dict(), hnode[1].to_dict()]

[{'id_': '91619bb2-1168-4134-a89e-56763bfb6b4c',
  'embedding': None,
  'metadata': {},
  'excluded_embed_metadata_keys': [],
  'excluded_llm_metadata_keys': [],
  'relationships': {<NodeRelationship.SOURCE: '1'>: {'node_id': '1b29b151-62a0-42d8-824d-33fb9d54a3f3',
    'node_type': <ObjectType.DOCUMENT: '4'>,
    'metadata': {},
    'hash': '1c97fa7e5825b66afd171e255435b693c2740279960611e2642c5a3f88c56488',
    'class_name': 'RelatedNodeInfo'},
   <NodeRelationship.PREVIOUS: '2'>: {'node_id': '7a476ec2-7cfa-48e2-8ed0-e21d52e74b5e',
    'node_type': <ObjectType.TEXT: '1'>,
    'metadata': {},
    'hash': '661c46b1a69e1ba2c285e52addef24d2252276a6cbceebd614234030233a9542',
    'class_name': 'RelatedNodeInfo'},
   <NodeRelationship.NEXT: '3'>: {'node_id': '76353e33-fa97-4936-acce-97b653289fe1',
    'node_type': <ObjectType.TEXT: '1'>,
    'metadata': {},
    'hash': '906cf07e5d02bb576d16a68ae62230d7d7d39c3a66791dd614a3193048dab023',
    'class_name': 'RelatedNodeInfo'}},
  'text': '1. ◎请关注

In [6]:
for chunk_size in [128, 256, 512, 1024]:
    index_name = f"chunk-{chunk_size}"
    index_dict[index_name] = create_index(chunk_size, index_name)

In [16]:
from testcase import question_list
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
for q in question_list:
    print(f"{bcolors.BOLD}{bcolors.HEADER}Q: {q}{bcolors.ENDC}")
    for chunk_size in [128, 256, 512, 1024]:
        key = f"chunk-{chunk_size}"
        print(f"{bcolors.HEADER}{key}{bcolors.ENDC}")
        retriever_engine = index_dict[key].as_retriever()
        results = retriever_engine.retrieve(q)
        for node in results[:5]:
            print(f"{bcolors.BOLD}Score: {node.score}{bcolors.ENDC}")
            print(node.text)
        

Q: 在巴塞罗那可以体验哪些有趣的事情
chunk-128
Score: 0.89052486

在巴塞罗那，我一直很喜欢世界博览会的那一片叫做蒙特惠奇山（Montjuic）的地方。
Score: 0.8877646

我们后来也在马德里的拉布拉斯大道上走过，可是都没有巴塞罗那这里好玩。这里热热闹闹，挤满了小摊贩、小酒铺、小吃摊，当然，还有各路大显身手的艺人。最精彩的是，它结合了这一片起于罗马时代的老城。
chunk-256
Score: 0.8886671

可惜我们没有时间能像在巴塞罗那一样，好好在马德里的老城区逛逛。在马德里扩建的新城区，也有一些很好玩的地方。例如塞万提斯的纪念碑。

在十六七世纪的风云跌宕起伏中，难得有一个平民，活得像塞万提斯那样，堪称是整整一本西班牙的二百年传奇。
Score: 0.87370956

这里热热闹闹，挤满了小摊贩、小酒铺、小吃摊，当然，还有各路大显身手的艺人。最精彩的是，它结合了这一片起于罗马时代的老城。拉布拉斯大道的一端，抵达地中海海边的海港，一路走下去，一直可以走到深蓝色的海水边。那里，耸入云霄的纪念柱上，站着哥伦布的金色雕像。拉布拉斯大道的另一端，就是著名的加泰罗尼亚广场。二十世纪巴塞罗那的很多故事发生在这个广场上。

![image](./Images/004.
chunk-512
Score: 0.8765443

二十世纪巴塞罗那的很多故事发生在这个广场上。

![image](./Images/004.jpeg)

享受街头喝啤酒的美国年轻人（这在美国是违禁的）

我们两次来巴塞罗那，都住过老城。这次，我们拖着行李从大道的起点，顺着步行道的中间，走了整整两站地铁的距离，只是为了感受一下这里的欢乐。一个小酒铺前，桌子拼在一起，变成长长的一溜。周围坐了一圈兴高采烈的美国大男孩，看到他们，我们忍不住会心笑起来，因为他们每人手里端着一个特大号的啤酒杯。照美国的法律，在大街上这么喝酒是不容许的。他们兴奋不已，只是因为他们可以感受“合法地在大街上喝酒”的刺激，再说，还是以那么开心的方式。我们说，让我们拍张照啊，他们很助兴，齐齐地高高举起了冒着泡沫的大玻璃杯。我们一路走一路给艺人们打分，最后一致同意，“最佳创意”颁给了两个和骷髅一起骑自行车的艺人。
Score: 0.87324184

花一天青年旅舍的钱，在这里可以住三天。

In [2]:
index_name = "chunk-128-512-2048"
vector_store = get_vector_store(index_name)
index = get_index(vector_store, llm, embed_model)
for chunk_size in [128, 512, 2048]:
    ssp = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=20)
    index.build_index_from_nodes(ssp.get_nodes_from_documents(document))
from testcase import question_list
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
for q in question_list:
    print(f"{bcolors.BOLD}{bcolors.HEADER}Q: {q}{bcolors.ENDC}")
    retriever_engine = index.as_retriever()
    results = retriever_engine.retrieve(q)
    for node in results[:5]:
        print(f"{bcolors.BOLD}Score: {node.score}{bcolors.ENDC}")
        print(node.text)

Q: 在巴塞罗那可以体验哪些有趣的事情
Score: 0.89052486
在巴塞罗那，我一直很喜欢世界博览会的那一片叫做蒙特惠奇山（Montjuic）的地方。
Score: 0.8877646
我们后来也在马德里的拉布拉斯大道上走过，可是都没有巴塞罗那这里好玩。这里热热闹闹，挤满了小摊贩、小酒铺、小吃摊，当然，还有各路大显身手的艺人。最精彩的是，它结合了这一片起于罗马时代的老城。
Q: 阿尔罕宫的有趣历史故事
Score: 0.8913774
[image](../Images/087.jpeg)

阿尔汉布拉宫城堡

也许，我们曾经和欧文走进同一个花园。我们看到的是那些姹紫嫣红、精心修剪的花园，而他看到的只是荒草萋萋中残留的花朵，可是他却更真切地触摸到了历史。可贵的是，欧文不仅因来自新大陆，而对欧洲纷争的历史保持着距离，而且他也没有对某一种宗教持有好恶的偏见。他只是对这个宫廷发生的故事，怀着几近天真的好奇细细探究，也对失败的一方怀着同情。对华盛顿·欧文来说，格拉那达摩尔人王朝的终结，有着一种历史宿命的悲怆感觉。

就在这里，华盛顿·欧文徘徊在阿尔汉布拉紧闭的深宫里。半夜，他被无名的声响惊醒，举着烛火，独自一人寻找传说中的游魂。他终于探明，传说中夜间鬼魂的幽幽哭泣，其实是阿尔汉布拉宫里摩尔人设计的复杂的水管系统中，流水冲击铅管发出的声响。白天，他常常下山去城里耶稣会的图书馆，细细查阅那里的档案，找出摩尔人传说的历史真相。

阿尔汉布拉宫是军事防御和宫廷的结合，其中有城堡和兵营的遗迹。在那里没有“皇宫”的感觉。我们攀上高高的城堡，很久不想下来，下面是叫做阿尔巴辛（Albaxin）的小镇。它是格拉那达的郊区。从高处看下去，小镇特别美。我们站的地方下面，正对着一个修女院。红色地砖的内院，洁白的墙，土红色的瓦，整个建筑群简洁又丰富。一个穿着黑白装的修女在廊里款款地走过。

![image](../Images/088.jpeg)

阿尔巴辛的修道院

阿尔汉布拉宫的另一部分要走一段路，是十三世纪修的一个花园，叫做“杰那拉里夫”。那里以绿色侧柏为墙，映衬着各色玫瑰。内庭院是阿拉伯建筑之所长，在封闭空间中创造出开敞空旷和阳光明朗。经典的长长的阿拉伯水池，两排细致的喷泉面对面，一对对地在空中交叉，好像是水的芭蕾。庭院是简洁的，端头却是美极了的满覆精细雕饰的柱廊，色调沉稳。这色调

WoW, seems mix different chunk size make retrieved results better, and there is no overlap happened.